In [ ]:
if "logger" in globals() and  len(logger.handlers) >= 1:
    # To suppress debug output from the logger defined in .ipython/profile_default/ipython_config.py
    logger.handlers.pop()

%load_ext autoreload
%autoreload 2

%matplotlib inline

import platform
BASEDIR = "/u/timoh/phd" if "freya" in platform.node() else ""

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
display(HTML("<style>.output_html { display: block; margin-left: auto; margin-right: auto; }</style>"))

In [ ]:
import os
import sys
import numpy
import scipy
from matplotlib import pyplot
pyplot.style.use("tlrh")

if "/tidalshocks/src" not in sys.path:
    sys.path.insert(0, "{}/tidalshocks/src".format(BASEDIR))
from mw_gc_observation import MwGcObservation

if "/supaharris" not in sys.path: 
    sys.path.insert(0, "{}/supaharris".format(BASEDIR))
from utils import parsec2arcmin

import logging
logging.getLogger("keyring").setLevel(logging.CRITICAL)
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)
logging.basicConfig(stream=sys.stdout, format="%(message)s")
logger = logging.getLogger(); logger.level = logging.INFO
for h in logger.handlers: h.setLevel(logger.level)
    
from amuse.units import units
from amuse.units import constants
from amuse.io import read_set_from_file

from amuse_wrapper import get_particle_properties
from test_stability_in_isolation import plot_diagnostics
from test_stability_in_isolation import analyse_snapshot
from test_stability_in_isolation import analyse_isolation

In [ ]:
Nstars = 5000
for gc_name in ["NGC 104"]:
    obs = MwGcObservation(logger, gc_name)
    limepy_model, limepy_sampled, amuse_sampled, converter = \
        obs.sample_deBoer2019_bestfit_king(Nstars=Nstars, verbose=True)

    fig = plot_diagnostics(obs, limepy_model, amuse_sampled, model_name="king", Tsnap=None, 
            softening=0.1, rmin=1e-3, rmax=1e3, Nbins=None, fig=None)
    pyplot.show()

# Analyse a single snapshot 

In [ ]:
%matplotlib inline
pyplot.style.use("tlrh")

obs = MwGcObservation(logger, "NGC 104")
# analyse_isolation(obs, "king", 1000, 0.1, 1337)

model_name = "king"
Nstars = 1000
softening = 0.1
seed = 1337
folder = "{}_{}_isolation_{}_{}_{}".format(obs.gc_slug, model_name, Nstars, softening, seed)
snap_base = "{}{}/{}_{}_isolation_{}_{}_{}_*.h5".format(obs.outdir, folder,
    obs.gc_slug, model_name, Nstars, softening, seed)
snapshots = sorted(glob.glob(snap_base), key=lambda s:
    [ int(c) for c in re.split('(\d+)', s) if c.isdigit()]
)
Nsnaps = len(snapshots)
print("\nFound {0} snapshots".format(Nsnaps))

args = []
kwargs = {
    "obs": obs,
    "model_name": model_name,
    "limepy_model": limepy_model,
    "isolation": True,
    "softening": softening,
    "seed": seed,
}

for i_fname_tuple in enumerate(snapshots):
    analyse_snapshot(i_fname_tuple, *args, **kwargs)
    break

# Analyse all snapshots for a particular simulation 

In [ ]:
obs = MwGcObservation(logger, "NGC 104")
# analyse_isolation(obs, "king", 1000, 0.1, 1337)